<a href="https://colab.research.google.com/github/ubcb/deepseek/blob/main/deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [45]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

login(hf_token)

In [46]:
import wandb

wb_token = userdata.get('WB_TOKEN')

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B sql to text',    # 设置项目名称 - 这里是用于SQL分析的DeepSeek模型微调项目
    job_type="training",
    anonymous="allow"# 允许匿名访问 # "allow"表示即使没有wandb账号的用户也能查看这个项目
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [47]:
# 从unsloth库中导入FastLanguageModel类
# unsloth是一个优化的语言模型加载和训练库
from unsloth import FastLanguageModel

# 设置模型参数
# 最大序列长度，即模型能处理的最大token数量
max_seq_length = 2048
dtype = None # 数据类型设置为None，让模型自动选择合适的数据类型
load_in_4bit = True # 启用4bit量化加载 # 4bit量化可以显著减少模型内存占用，但可能略微影响模型性能

# 加载预训练模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,     # 设置最大序列长度
    dtype = dtype,# 设置数据类型
    load_in_4bit = load_in_4bit,  # 启用4bit量化加载
    token = hf_token, # 使用Hugging Face的访问令牌
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [48]:

prompt = """Below is SQL query. Think like sql expert and generate a summary of the query which explains the use case of the query. As in
what the query is trying to read from the database in a usecase sense.

### Query:
SELECT
    c.customer_id,
    c.name AS customer_name,
    COUNT(o.order_id) AS total_orders,
    SUM(o.total_amount) AS total_spent,
    AVG(o.total_amount) AS avg_order_value,
    MAX(o.order_date) AS last_order_date
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
LEFT JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_date >= '2024-01-01'
GROUP BY c.customer_id, c.name
ORDER BY total_spent DESC
LIMIT 10;

### Response:
"""

In [7]:
# 将模型切换到推理模式
FastLanguageModel.for_inference(model)
# 对输入文本进行分词和编码
# [prompt] - 将prompt放入列表中，因为tokenizer期望批处理输入
# return_tensors="pt" - 返回PyTorch张量格式
# to("cuda") - 将张量移动到GPU上进行计算
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

# 使用模型生成响应
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
# 后处理生成的输出
# batch_decode - 将标记ID序列解码回文本
# split("### Response:")[1] - 提取"### Response:"之后的部分，即模型的实际回答
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])

#微调前：模型能给出基本的查询解释，但不够精确


The query retrieves data from a database to analyze customer information and their associated orders. It selects details for each customer, including their name, total orders, total spent, average order value, and the last order date. The query joins multiple tables to gather related data and filters based on the order date. It groups the results by customer and sorts them by the total amount spent in descending order, limiting the results to the top 10 customers. This allows for identifying high-spending customers for marketing or analysis purposes.

Okay, so I'm looking at this SQL query, and I need to figure out what it's trying to do. Let me break it down step by step.

First, the SELECT clause is selecting several columns from the customers table. It's taking customer_id, renaming name to customer_name for clarity, and then some aggregate functions on the orders table. So, it's pulling individual customer info along with their order data.

Next, the FROM clause is joining custome

In [49]:
#提示词
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a SQL expert with advance understanding of SQL queries. You can understand database schema from the query. Think like sql expert and generate a summary of the query which explains the use case of the query. As in
what the query is trying to read from the database in a usecase sense.

### Query:
{}

### Response:
<think>
{}
</think>
{}"""

In [50]:
!pip install datasets
!wget "https://huggingface.co/datasets/b-mc2/sql-create-context/resolve/main/sql_create_context_v4.json"

--2025-02-23 14:45:59--  https://huggingface.co/datasets/b-mc2/sql-create-context/resolve/main/sql_create_context_v4.json
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.118, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/f9/26/f926cb4f40d6cfd6696e03dcc169e95f1b3b9a8e8f965d6040d4bd5f07448faa/280961458ccb74731104ae9af715797a0332327a33a210120a6884085a5c03f0?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27sql_create_context_v4.json%3B+filename%3D%22sql_create_context_v4.json%22%3B&response-content-type=application%2Fjson&Expires=1740325559&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MDMyNTU1OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9mOS8yNi9mOTI2Y2I0ZjQwZDZjZmQ2Njk2ZTAzZGNjMTY5ZTk1ZjFiM2I5YThlOGY5NjVkNjA0MGQ0YmQ1ZjA3NDQ4ZmFhLzI4MDk2MTQ1OGNjYjc0NzMxMTA0YW

In [51]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/sql_create_context_v4.json", split="train[0:500]")
# - split: 指定要加载的数据集切片
#   - "train[0:500]" 表示只加载训练集的前500条数据
#   - 这种切片方式可以用于快速实验和调试


In [52]:
#处理数据集
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

#原数据集：输入是英文描述，输出是SQL反转后：输入是SQL(examples["answer"])，输出是英文描述(examples["question"])
def switch_and_format_prompt(examples):
    inputs = examples["answer"] # 使用 answer(SQL) 作为输入
    context = examples["context"]
    outputs = examples["question"] # 使用 question(英文描述) 作为输出
    texts = []
    for input, context, output in zip(inputs, context, outputs):
        text = train_prompt_style.format(input, context, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

# 应用转换
dataset = dataset.map(switch_and_format_prompt, batched = True)

In [54]:
#设置参数

model = FastLanguageModel.get_peft_model(
    model,
    r=16,   # LoRA的秩(rank)值，决定了低秩矩阵的维度，较大的r值(如16)可以提供更强的模型表达能力，但会增加参数量和计算开销，较小的r值(如4或8)则会减少参数量，但可能影响模型性能，通常在4-16之间选择,需要在性能和效率之间权衡
    target_modules=[ #指定需要应用LoRA微调的模块列表，q_proj, k_proj, v_proj: 注意力机制中的查询、键、值投影层
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj", #注意力输出投影层
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16, #缩放参数，用于控制LoRA更新的强度，通常设置为与r相同的值，较大的alpha会增加LoRA的影响力，较小的alpha则会减弱LoRA的影响
    lora_dropout=0, #LoRA层的dropout率，0表示不使用dropout，增加dropout可以帮助防止过拟合，但可能影响训练稳定性，在微调时通常设为0或很小的值
    bias="none", #是否微调偏置项，"none"表示不微调偏置参数，也可以设置为"all"或"lora_only"来微调不同范围的偏置
    use_gradient_checkpointing="unsloth",  # 梯度检查点策略，"unsloth"是一种优化的检查点策略，适用于长上下文可以显著减少显存使用，但会略微增加计算时间对处理长文本特别有用
    random_state=3407, #随机数种子，控制初始化的随机性，固定种子可以确保实验可重复性
    use_rslora=False, #是否使用RSLoRA(Rank-Stabilized LoRA) False表示使用标准LoRARSLoRA是一种改进的LoRA变体，可以提供更稳定的训练
    loftq_config=None, #LoftQ配置None表示不使用LoftQ量化LoftQ是一种用于模型量化的技术，可以减少模型大小
)

AttributeError: 'PeftModelForCausalLM' object has no attribute 'base_model'

In [57]:
#训练参数
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    ## 训练参数配置
    args=TrainingArguments(
        # 批处理相关
        per_device_train_batch_size=2, # 每个设备（GPU）的训练批次大小
        gradient_accumulation_steps=4,# 梯度累积步数，用于模拟更大的批次大小
         # 训练步数和预热
        warmup_steps=5,# 学习率预热步数，逐步增加学习率
        max_steps=60,# 最大训练步数
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(), # 如果不支持 bfloat16，则使用 float16
        bf16=is_bfloat16_supported(),# 如果支持则使用 bfloat16，通常在新型 GPU 上性能更好
        logging_steps=10,# 每10步记录一次日志
        optim="adamw_8bit", # 使用8位精度的 AdamW 优化器
        weight_decay=0.01,# 权重衰减率，用于防止过拟合
        lr_scheduler_type="linear",# 学习率调度器类型，使用线性衰减
        seed=3407,# 随机种子，确保实验可重复性
        output_dir="outputs", # 模型和检查点的输出目录
    ),
)


Tokenizing train dataset (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details